In [1]:
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
LCEL-Advanced


# 사용자 정의 함수를 실행하는 방법

In [4]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

def length_function(text):  # 텍스트의 길이를 반환하는 함수
    return len(text)


def _multiple_length_function(text1, text2):  # 두 텍스트의 길이를 곱하는 함수
    return len(text1) * len(text2)


def multiple_length_function(  # 2개 인자를 받는 함수로 연결하는 wrapper 함수
    _dict,
):  # 딕셔너리에서 "text1"과 "text2"의 길이를 곱하는 함수
    return _multiple_length_function(_dict["text1"], _dict["text2"])

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

# 프롬프트와 모델을 연결하여 체인 생성
chain1 = prompt | model

# 체인 구성
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
# 주어진 인자들로 체인을 실행합니다.
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 = 12'

# RunnableConfig 인자로 활용

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json

def parse_or_fix(text: str, config: RunnableConfig):
    # 다음 텍스트를 수정하는 프롬프트 템플릿을 생성
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\ntext\n{input}\n\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )

    # 최대 3번 시도
    for _ in range(3):
        try:
            # JSON 형식으로 텍스트를 파싱
            return json.loads(text)
        except Exception as e:
            # 파싱 중 오류가 발생하면 수정 체인을 호툴하여 텍스트를 수정
            text = fixing_chain.invoke({"input": text, "error": e}, config)
            print(f"config: {config}")
    # 파싱에 실패한 경우 "Failed to parse" 문자열을 반환
    return "Failed to parse"

In [10]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    # RunnableLambda를 사용하여 parse_or_fix 함수를 호출
    output = RunnableLambda(parse_or_fix).invoke(
        input="{foo:: bar}",
        config={"tags": ["my-tag"], "callbacks": [cb]}, # config를 전달
    )
    # 수정한 결과를 출력
    print(f"\n\n수정한 결과:\n{output}")

config: {'tags': ['my-tag'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x00000102979A5410>, 'recursion_limit': 25, 'configurable': {}}


수정한 결과:
{'foo': 'bar'}


In [11]:
output

{'foo': 'bar'}